# AS-AD model for the closed economy with static inflation expectations
The AS-AD model for the closed economy with static inflation expectations can be expressed in two equations: 
\begin{align}
\pi_t&=\pi_{t-1}+\gamma\left(y_t-\bar{y}\right)+s_t \tag{AS} \\
y_t-\bar{y}&=-\alpha\left(\pi_t-\pi^*\right)+z_t, \quad \alpha \equiv \frac{\alpha_2 h}{1+\alpha_2 b}  \tag{AD}
\end{align}



## User Guide
- Step 1: Click on the ***Runtime (Danish: Køreselstid)*** menu at the top and then ***Run all (Kør alle)***
- Step 2: Scroll down to the section ***Interactive plots*** to find the interactive plots*
- Step 3: You can now play around with the different sliders to investiage the effects of model parameters

## Importing packages

In [84]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Label, interactive_output
from IPython.display import display
import ipywidgets as widgets

## Plotting function

In [85]:
# Defining the AD curve
def AD(alpha2, h, b, ybar, pistar, z, grid):
    alpha = (alpha2*h)/(1+alpha2*b)
    AD = pistar - 1/alpha * (grid - ybar) + 1/alpha*z
    return AD
    
# Defining the AS curve
def AS(pibef, gamma, ybar, s, grid):
    AS = pibef + gamma*(grid - ybar) + s
    return AS

# Computing the equilibrium in each period
def equilibrium(alpha2, h, b, gamma, ybar, pistar, pibef, z, s):
    alpha = (alpha2*h)/(1+alpha2*b)
    y_eq = alpha / (1+alpha*gamma) * (pistar + 1/alpha*z - pibef - s) + ybar
    pi_eq = pibef + gamma*alpha/(1 + gamma*alpha) * (pistar+1/alpha*z-pibef-s) + s
    return y_eq, pi_eq

def AS_solve(alpha2, h, b, gamma, ybar, pistar, z, s, N, grid):
#     z, s = np.array(z_inp), np.array(s_inp)
    y_eq, pi_eq = np.zeros([N]), np.zeros([N])
    AS_matrix, AD_matrix = np.zeros([len(grid), N]), np.zeros([len(grid), N])
    y_eq[0], pi_eq[0] = ybar, pistar
    AS_matrix[:,0], AD_matrix[:,0] = AS(pistar, gamma, ybar, 0, grid), AD(alpha2, h, b, ybar, pistar, 0, grid)
    for n in range(1,N):
            if n == 1: 
                y_eq[n], pi_eq[n] = equilibrium(alpha2, h, b, gamma, ybar, pistar, pi_eq[n-1], z, s)
                AS_matrix[:,n] = AS(pi_eq[n-1], gamma, ybar, s, grid)
                AD_matrix[:,n] = AD(alpha2, h, b, ybar, pistar, z, grid)
            else:
                y_eq[n], pi_eq[n] = equilibrium(alpha2, h, b, gamma, ybar, pistar, pi_eq[n-1], 0, 0)
                AS_matrix[:,n] = AS(pi_eq[n-1], gamma, ybar, 0, grid)
                AD_matrix[:,n] = AD(alpha2, h, b, ybar, pistar, 0, grid)                
    return y_eq, pi_eq, AS_matrix, AD_matrix

def IRF(alpha2, h, b, gamma, ybar, pistar, z, s, T):
    y_hat, pi_hat = np.zeros([T+1]), np.zeros([T+1])
    y_hat[0], pi_hat[0] = ybar, pistar
    for t in range(1,T+1):
            if t == 1:
                y_hat[t], pi_hat[t] = equilibrium(alpha2, h, b, gamma, ybar, pistar, pi_hat[t-1], z, s)
            else:
                y_hat[t], pi_hat[t] = equilibrium(alpha2, h, b, gamma, ybar, pistar, pi_hat[t-1], 0, 0)
    y_hat = y_hat - ybar
    pi_hat = pi_hat - pistar
    return y_hat, pi_hat

Coding the plotting function for the interactive AS-AD diagram

In [86]:
def inter_plot(alpha2, h, b, gamma, ybar, pistar, z, s, N, grid):
    y_seq, pi_seq, AS_seq, AD_seq = AS_solve(alpha2, h, b, gamma, ybar, pistar, z, s, N, grid)
    fig, ax1 = plt.subplots(figsize=(12, 6), dpi = 200)

    # Plot on primary y-axis
    ax1.scatter(y_seq, pi_seq, label='AD', color = 'black', zorder=4)
    ax1.plot(grid, AS_seq, label='AS', color = 'red')
    ax1.plot(grid, AD_seq[:,0:2], label='AD', color = 'blue')
    ax1.set_xlabel('Output (y)', fontsize = 12)
    ax1.set_ylabel('Inflation ($\pi$)', fontsize = 12)
    ax1.grid(True, zorder=0, linestyle = '--')
    ax1.set_ylim((0, int(max(pi_seq)+2)))
    ax1.set_xlim((grid[0], int(max(y_seq)+2)))
    x_coordinate = ybar
    plt.axvline(x=x_coordinate, color='r', linestyle='-', label='Vertical Line at x={}'.format(x_coordinate))
    plt.text(ybar+0.05, int(max(pi_seq))+1.75, f'$LRAS$',fontsize = 14)
    
    for i in range(N):
        plt.annotate("",xy=(y_seq[i], pi_seq[i]), xytext=(y_seq[i], 0), arrowprops=dict(width=0.1, color='grey', headwidth=0.01, headlength=0.01))
        plt.annotate("",xy=(y_seq[i], pi_seq[i]), xytext=(0, pi_seq[i]), arrowprops=dict(width=0.1, color='grey', headwidth=0.01, headlength=0.01))

    for i in range(1,N):
        plt.text(y_seq[i]-0.03, pi_seq[i]+0.08, f'$E_{i}$',fontsize = 10)
        
    plt.text(y_seq[0]+0.01, 0.05, r'$\bar{y}$',fontsize = 10)
    plt.text(0.01, pi_seq[0]+0.02, '$\pi^*$',fontsize = 10)
    for i in range(1,N):
        plt.text(y_seq[i]+0.01, 0.08, f'$y_{i}$',fontsize = 10)
        plt.text(0.01, pi_seq[i]+0.05, f'$\pi_{i}$',fontsize = 10)

    lines, labels = ax1.get_legend_handles_labels()
    plt.title("AS-AD Diagram with static expectations", fontsize = 16) 

    plt.show() 

Coding the plotting function for the interactive impulse response functions

In [87]:
def irf_inter_plot(alpha2, h, b, gamma, ybar, pistar, z, s, T):
    T_grid = [t for t in range(T+1)]
    y_hat, pi_hat = IRF(alpha2, h, b, gamma, ybar, pistar, z, s, T)
    
    fig, ax1 = plt.subplots(figsize=(12, 6), dpi = 400)

    # Plot on primary y-axis
    ax1.plot(T_grid, y_hat, label=r'Output gap $(\hat{y}_t \equiv y_t - \bar{y})$', color = 'red')
    ax1.plot(T_grid, pi_hat, label=r'Inflation gap $(\hat{\pi}_t\equiv \pi_t - \pi^*))$', color = 'blue')
#     ax1.plot(T_grid, er, label=r'Real Exchange Rate $(e^r)$', color = 'orange')
    ax1.set_xlabel('Horizon', fontsize = 12)
    ax1.set_ylabel('Deviation from steady state', fontsize = 12)
    ax1.set_xlim((T_grid[0], T_grid[-1]))
    ax1.grid(True, zorder=0, linestyle = '--')

    # Add legend
    lines, labels = ax1.get_legend_handles_labels()
    ax1.legend(lines, labels, loc='upper right', fontsize = 12)
    plt.title("IRFs for Open Economy AS-AD model with Fixed Exchange Rate", fontsize = 16) 

    plt.show() 

# Interactive plots

This cell just creates all the sliders needed for the plot to work (looks involed but is very repetitive)

In [91]:
# Creating sliders for different model parameters
z_slider = widgets.FloatSlider(value=1,min=-4.0,max=4.0,step=1.0,description=r"$z$",continuous_update=False)
s_slider = widgets.FloatSlider(value=0,min=-4.0,max=4.0,step=1.0,description=r"$s$",continuous_update=False)
alpha2_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$\alpha_2$",continuous_update=False)
gamma_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$\gamma$",continuous_update=False)
h_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$h$",continuous_update=False)
b_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$b$",continuous_update=False)
ybar_slider = widgets.FloatSlider(value=2,min=0.0,max=4.0,step=0.5,description=r"$\bar{y}$",continuous_update=False)
pistar_slider = widgets.FloatSlider(value=2,min=0.0,max=4.0,step=0.5,description=r"$\pi^*$",continuous_update=False)
N_slider = widgets.IntSlider(value=3,min=1,max=5,step=1,description=r"$N$",continuous_update=False)
T_slider = widgets.IntSlider(value=8,min=1,max=15,step=1,description=r"$T$",continuous_update=False)

# Arrange sliders in two columns using VBox and HBox
left_box = widgets.VBox([z_slider, s_slider, alpha2_slider])
center_box = widgets.VBox([gamma_slider, h_slider, b_slider])
right_box_asad = widgets.VBox([ybar_slider, pistar_slider, N_slider])
hbox_asad = widgets.HBox([left_box, center_box,right_box_asad])

right_box_irf = widgets.VBox([ybar_slider, pistar_slider, N_slider])
hbox_irf = widgets.HBox([left_box, center_box,right_box_irf])

## AS-AD diagram

In [92]:
# Create the interactive AS-AD diagram
plot_output = interactive_output(inter_plot, {'alpha2': alpha2_slider, 'gamma': gamma_slider, 'ybar': ybar_slider, 
                                              'pistar': pistar_slider, 'z': z_slider, 's': s_slider, 'N': N_slider, 
                                              'grid': fixed(np.linspace(0, 10, 100)), 'b': b_slider, 'h': h_slider})
display(hbox_asad, plot_output)
plot_output.layout.height = '550px'

Output()

## Impulse-response functions

In [93]:
# Create the interactive plot
plot_output = interactive_output(irf_inter_plot, {'alpha2': alpha2_slider, 'h': h_slider, 'b': b_slider, 
                                                  'gamma': gamma_slider, 'ybar': ybar_slider, 'pistar': pistar_slider, 
                                                  'z': z_slider, 's': s_slider, 'T': T_slider})
display(hbox_irf, plot_output)
plot_output.layout.height = '550px'

Output()